# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235847917872                   -0.50    7.0   11.8ms
  2   -7.250327938398       -1.84       -1.40    1.0   7.67ms
  3   -7.250999559449       -3.17       -1.90    2.0   8.51ms
  4   -7.250948696699   +   -4.29       -1.87    2.0   9.06ms
  5   -7.251331068999       -3.42       -2.67    1.0   7.37ms
  6   -7.251338293023       -5.14       -3.29    1.0   6.55ms
  7   -7.251338734100       -6.36       -3.65    2.0   6.33ms
  8   -7.251338781367       -7.33       -3.93    1.0   5.41ms
  9   -7.251338795389       -7.85       -4.29    2.0   6.17ms
 10   -7.251338798198       -8.55       -4.79    2.0   6.30ms
 11   -7.251338798679       -9.32       -5.24    2.0   6.57ms
 12   -7.251338798697      -10.73       -5.64    1.0   5.71ms
 13   -7.251338798704      -11.20       -6.14    2.0   6.66ms
 14   -7.251338798704      -12.08       -6.55    2.0   6.81ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03822593620238611
[ Info: Arnoldi iteration step 2: normres = 0.5640318937978195
[ Info: Arnoldi iteration step 3: normres = 0.9218079878776702
[ Info: Arnoldi iteration step 4: normres = 0.38624350882467695
[ Info: Arnoldi iteration step 5: normres = 0.24085755746152887
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.07e-02, 3.88e-02, 1.78e-01, 1.40e-01, 6.57e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4929826070254614
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.17e-02, 5.28e-02, 4.71e-01, 1.15e-01, 2.80e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08483570533764531
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.35e-04, 8.48e-03, 3.60e-02, 2.77e-02, 6.75e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10277346050423909
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.39e